# Data Processing

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
pd.set_option('display.max_columns', 100)

In [2]:
train_df = pd.read_csv('data/train.csv')
train_df.shape

(1460, 81)

In [3]:
train_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
with pd.option_context("display.max_rows", 1000):
    display(train_df.isna().sum().loc[train_df.isna().sum() > 0].sort_values(ascending=False))

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
Electrical         1
dtype: int64

In [5]:
train_df.isna().sum().loc[train_df.isna().sum() > 0].index


Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

## Plan for NaN

- LotFrontage      259: Iterative Imputer from LotArea
- Alley           1369: set to NA
- MasVnrType       872: set to None
- MasVnrArea         8: set to 0
- BsmtQual          37: set to NA
- BsmtCond          37: set to NA
- BsmtExposure      38: set to NA
- BsmtFinType1      37: set to NA
- BsmtFinType2      38: set to NA
- Electrical         1: set to most frequent
- FireplaceQu      690: Iterative Imputer from Fireplace
- GarageType        81: set to NA
- GarageYrBlt       81: set to NA
- GarageFinish      81: set to NA
- GarageQual        81: set to NA
- GarageCond        81: set to NA
- PoolQC          1453: Iterative Imputer from PoolArea
- Fence           1179: set to NA
- MiscFeature     1406: set to NA

In [6]:
fix_na = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'MiscFeature']
fix_none = ['MasVnrType']
fix_zero = ['MasVnrArea']
fix_most_frequent = ['Electrical']
fix_iterative = {'FireplaceQu': 'Fireplaces', 'PoolQC': 'PoolArea', 'LotFrontage': 'LotArea'}

In [7]:
fix_na_imputer = SimpleImputer(strategy='constant', fill_value='NA')
fix_na_imputer.fit(train_df[fix_na])
train_df[fix_na] = fix_na_imputer.transform(train_df[fix_na])

In [8]:
fix_none_imputer = SimpleImputer(strategy='constant', fill_value='None')
fix_none_imputer.fit(train_df[fix_none])
train_df[fix_none] = fix_none_imputer.transform(train_df[fix_none])

In [9]:
fix_zero_imputer = SimpleImputer(strategy='constant', fill_value=0)
fix_zero_imputer.fit(train_df[fix_zero])
train_df[fix_zero] = fix_zero_imputer.transform(train_df[fix_zero])

In [10]:
fix_most_frequent_imputer = SimpleImputer(strategy='most_frequent')
fix_most_frequent_imputer.fit(train_df[fix_most_frequent])
train_df[fix_most_frequent] = fix_most_frequent_imputer.transform(train_df[fix_most_frequent])

In [11]:
# feature, ranking
ordinals = {
    'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
    'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
    'LandSlope': ['Gtl', 'Mod', 'Sev'],
    'ExterQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'ExterCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'BsmtQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'BsmtCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'NA'],
    'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
    'BsmtFinType2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
    'HeatingQC': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
    'FireplaceQu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'GarageFinish': ['Fin', 'RFn', 'Unf', 'NA'],
    'GarageQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'GarageCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'PavedDrive': ['Y', 'P', 'N'],
    'PoolQC': ['Ex', 'Gd', 'TA', 'Fa', 'NA'],
    'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA']
}

for feature, ranking in ordinals.items():
    print(f"Processing {feature}...")
    encoder = OrdinalEncoder(categories=[ranking], handle_unknown="use_encoded_value", unknown_value=np.nan)
    train_df[feature] = encoder.fit_transform(train_df[[feature]])

Processing LotShape...
Processing LandContour...
Processing LandSlope...
Processing ExterQual...
Processing ExterCond...
Processing BsmtQual...
Processing BsmtCond...
Processing BsmtExposure...
Processing BsmtFinType1...
Processing BsmtFinType2...
Processing HeatingQC...
Processing KitchenQual...
Processing Functional...
Processing FireplaceQu...
Processing GarageFinish...
Processing GarageQual...
Processing GarageCond...
Processing PavedDrive...
Processing PoolQC...
Processing Fence...


In [12]:
set_conditional_imputer = IterativeImputer(initial_strategy='most_frequent')
for field, condition in fix_iterative.items():
    set_conditional_imputer.fit(train_df[[field, condition]])
    train_df[field] = np.round(set_conditional_imputer.transform(train_df[[field, condition]]))[:, 0]

In [13]:
train_df.isna().sum().loc[train_df.isna().sum() > 0]

Series([], dtype: int64)

In [14]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NA,0.0,0.0,AllPub,Inside,0.0,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,1.0,2.0,PConc,1.0,2.0,3.0,0.0,706,5.0,0,150,856,GasA,0.0,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,1.0,8,0.0,0,2.0,Attchd,2003.0,1.0,2,548,2.0,2.0,0.0,0,61,0,0,0,0,1.0,4.0,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NA,0.0,0.0,AllPub,FR2,0.0,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,2.0,2.0,CBlock,1.0,2.0,0.0,1.0,978,5.0,0,284,1262,GasA,0.0,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,2.0,6,0.0,1,2.0,Attchd,1976.0,1.0,2,460,2.0,2.0,0.0,298,0,0,0,0,0,1.0,4.0,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NA,1.0,0.0,AllPub,Inside,0.0,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,1.0,2.0,PConc,1.0,2.0,2.0,0.0,486,5.0,0,434,920,GasA,0.0,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,1.0,6,0.0,1,2.0,Attchd,2001.0,1.0,2,608,2.0,2.0,0.0,0,42,0,0,0,0,1.0,4.0,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NA,1.0,0.0,AllPub,Corner,0.0,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,2.0,2.0,BrkTil,2.0,1.0,3.0,1.0,216,5.0,0,540,756,GasA,1.0,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,1.0,7,0.0,1,1.0,Detchd,1998.0,2.0,3,642,2.0,2.0,0.0,0,35,272,0,0,0,1.0,4.0,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NA,1.0,0.0,AllPub,FR2,0.0,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,1.0,2.0,PConc,1.0,2.0,1.0,0.0,655,5.0,0,490,1145,GasA,0.0,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,1.0,9,0.0,1,2.0,Attchd,2000.0,1.0,3,836,2.0,2.0,0.0,192,84,0,0,0,0,1.0,4.0,NA,0,12,2008,WD,Normal,250000


In [15]:
train_df.select_dtypes(include=['object']).columns

Index(['MSZoning', 'Street', 'Alley', 'Utilities', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'CentralAir', 'Electrical', 'GarageType', 'GarageYrBlt',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')

In [16]:
nominals = ['MSZoning', 'Street', 'Alley', 'Utilities', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'CentralAir', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']

In [17]:
train_df.MSZoning.value_counts()

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

In [18]:
for nominal in nominals:
    print(f"Processing {nominal}...")
    encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
    encoded = encoder.fit_transform(train_df[[nominal]])
    # encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names([nominal]))
    train_df = pd.concat([train_df, encoded], axis=1).drop(columns=nominal)

Processing MSZoning...
Processing Street...
Processing Alley...
Processing Utilities...
Processing LotConfig...
Processing Neighborhood...
Processing Condition1...
Processing Condition2...
Processing BldgType...
Processing HouseStyle...
Processing RoofStyle...
Processing RoofMatl...
Processing Exterior1st...
Processing Exterior2nd...
Processing MasVnrType...
Processing Foundation...
Processing Heating...
Processing CentralAir...
Processing Electrical...
Processing GarageType...
Processing MiscFeature...
Processing SaleType...
Processing SaleCondition...


In [23]:
train_df.head()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,...,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,0.0,0.0,0.0,7,5,2003,2003,196.0,1.0,2.0,1.0,2.0,3.0,0.0,706,5.0,0,150,856,0.0,856,854,0,1710,1,0,2,1,3,1,1.0,8,0.0,0,2.0,2003.0,1.0,2,548,2.0,2.0,0.0,0,61,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,80.0,9600,0.0,0.0,0.0,6,8,1976,1976,0.0,2.0,2.0,1.0,2.0,0.0,1.0,978,5.0,0,284,1262,0.0,1262,0,0,1262,0,1,2,0,3,1,2.0,6,0.0,1,2.0,1976.0,1.0,2,460,2.0,2.0,0.0,298,0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,68.0,11250,1.0,0.0,0.0,7,5,2001,2002,162.0,1.0,2.0,1.0,2.0,2.0,0.0,486,5.0,0,434,920,0.0,920,866,0,1786,1,0,2,1,3,1,1.0,6,0.0,1,2.0,2001.0,1.0,2,608,2.0,2.0,0.0,0,42,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,60.0,9550,1.0,0.0,0.0,7,5,1915,1970,0.0,2.0,2.0,2.0,1.0,3.0,1.0,216,5.0,0,540,756,1.0,961,756,0,1717,1,0,1,0,3,1,1.0,7,0.0,1,1.0,1998.0,2.0,3,642,2.0,2.0,0.0,0,35,272,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,84.0,14260,1.0,0.0,0.0,8,5,2000,2000,350.0,1.0,2.0,1.0,2.0,1.0,0.0,655,5.0,0,490,1145,0.0,1145,1053,0,2198,1,0,2,1,4,1,1.0,9,0.0,1,2.0,2000.0,1.0,3,836,2.0,2.0,0.0,192,84,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
